In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time, json
import sys
sys.path.append("../")
import os
import torch
import numpy as np
from tqdm import tqdm

import logging
from src.utils import logging_utils
from src.utils import env_utils, experiment_utils
from src import functional
import wandb

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-10-29 14:19:18 __main__ INFO     torch.__version__='2.5.0+cu124', torch.version.cuda='12.4'


In [3]:
from src.models import ModelandTokenizer

# MODEL_KEY = "meta-llama/Llama-3.2-3B-Instruct"
# MODEL_KEY = "meta-llama/Llama-3.1-8B-Instruct"

MODEL_KEY = "meta-llama/Llama-3.2-3B"
# MODEL_KEY = "google/gemma-2-2b"
# MODEL_KEY = "meta-llama/Llama-3.1-8B"

#! torch.adaptive precision
mt = ModelandTokenizer(
    model_key=MODEL_KEY,
    torch_dtype=torch.float16,
)

2024-10-29 14:19:22 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.00s/it]

2024-10-29 14:19:24 src.models INFO     loaded model </home/local_arnab/Codes/00_MODEL/meta-llama/Llama-3.2-3B> | size: 6127.841 MB | dtype: torch.float16 | device: cuda:0


In [5]:
from src.dataset_manager import DatasetManager
dataloader = DatasetManager.from_named_datasets(
    [("geometry_of_truth", "cities")],
    batch_size=32
)

In [6]:
batch = next(iter(dataloader))

In [7]:
batch

[ContextQASample(context="The city of Ya'an is in Morocco.", questions=['# Would you validate this statement?', '# Do you find this to be truthful?', '# Do you find this to be truthful?', '# Have I understood this correctly?', '# Could you tell me if this is factual?', '# May I assume this is correct?', '# Would you validate this statement?', '# Are you confirming this is true?', '# Do you find this to be truthful?', '# Are you saying this is correct?'], answers=['No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No']),
 ContextQASample(context='The city of Xinyang is in Japan.', questions=['# Would you validate this statement?', '# Is this statement true?', '# Do you agree that this is true?', '# Can we confirm this is accurate?', '# Would you say this is accurate?', '# Can you verify if this is true?', '# Does this statement hold true?', '# Can we confirm this is accurate?', '# Does this statement hold true?', '# Does this information ring true?'], answers=['No', 'No', 'No', 'No'

In [4]:
from src.utils.typing import LatentCache, LatentCacheCollection
from src.tokens import prepare_input
from src.functional import get_module_nnsight, interpret_logits
from src.functional import get_batch_concept_activations

# prompts = [
#     "Eiffel Tower is in which city? It is in",
#     "A quick brown fox",
#     "The sun rises in the",
# ]

prompts = [
    "The land of the rising sun is",
    "The capital of France is",
    "The Space Needle is a tower in"
]

latents = get_batch_concept_activations(
    mt=mt,
    prompts=prompts,
    interested_layer_indices=list(range(5, 20)),
    check_prediction=None,
    on_token_occur=None,
)

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [5]:
lcc = LatentCacheCollection(latents=latents)
lcc.detensorize()

In [7]:
from src.dataset_manager import DatasetManager

# group, name = "relations", "factual/country_capital_city"
group, name = "relations", "commonsense/fruit_outside_color"

dataloader = DatasetManager.from_named_datasets(
    [
        (group, name),
    ],
    batch_size=8
)

2024-10-28 23:57:21 numexpr.utils INFO     Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-10-28 23:57:21 numexpr.utils INFO     NumExpr defaulting to 8 threads.
2024-10-28 23:57:21 datasets INFO     PyTorch version 2.5.0 available.
2024-10-28 23:57:21 src.dataset_manager INFO     Loaded 60 examples from commonsense/fruit_outside_color.


In [8]:
cache_dir = os.path.join(group, name)
os.makedirs(cache_dir, exist_ok=True)

for batch_idx, batch in tqdm(enumerate(dataloader)):
    prompts = [b.context for b in batch]    
    latents = get_batch_concept_activations(
        mt=mt,
        prompts=prompts,
        interested_layer_indices=list(range(5, 20)),
        check_prediction=None,
        on_token_occur=None,
    )

    correct_labels = [b.correct for b in batch]
    incorrect_labels = [b.incorrect for b in batch]

    for latent_cache, correct, incorrect in zip(latents, correct_labels, incorrect_labels):
        latent_cache.correct_label = correct
        latent_cache.incorrect_label = incorrect
        latent_cache.group="relations"
    
    lcc = LatentCacheCollection(latents=latents)
    lcc.detensorize()

    with open(os.path.join(cache_dir, f"batch_{batch_idx}.json"), "w") as f:
        f.write(lcc.to_json())

0it [00:00, ?it/s]

0it [00:00, ?it/s]


AttributeError: 'ContextQASample' object has no attribute 'correct'

In [48]:
from src.activation_manager import ActivationLoader

In [13]:
from src.activation_manager import ActivationLoader, get_batch_paths

latent_root = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR, 
    "cached_latents", MODEL_KEY.split("/")[-1], "geometry_of_truth"
)

activation_batch_paths = list(get_batch_paths(latent_root))
activation_batch_paths

['/home/local_arnab/Codes/Projects/talkative_probes/results/cached_latents/Llama-3.2-3B/geometry_of_truth/cities/batch_6.json',
 '/home/local_arnab/Codes/Projects/talkative_probes/results/cached_latents/Llama-3.2-3B/geometry_of_truth/cities/batch_41.json',
 '/home/local_arnab/Codes/Projects/talkative_probes/results/cached_latents/Llama-3.2-3B/geometry_of_truth/cities/batch_42.json',
 '/home/local_arnab/Codes/Projects/talkative_probes/results/cached_latents/Llama-3.2-3B/geometry_of_truth/cities/batch_34.json',
 '/home/local_arnab/Codes/Projects/talkative_probes/results/cached_latents/Llama-3.2-3B/geometry_of_truth/cities/batch_12.json',
 '/home/local_arnab/Codes/Projects/talkative_probes/results/cached_latents/Llama-3.2-3B/geometry_of_truth/cities/batch_17.json',
 '/home/local_arnab/Codes/Projects/talkative_probes/results/cached_latents/Llama-3.2-3B/geometry_of_truth/cities/batch_28.json',
 '/home/local_arnab/Codes/Projects/talkative_probes/results/cached_latents/Llama-3.2-3B/geometry_o

In [17]:
act_loader = ActivationLoader(
    latent_cache_files=activation_batch_paths,
    shuffle=True,
    batch_size=32,
)

In [18]:
act_batch = act_loader.next_batch()

In [19]:
act_batch

[ActivationSample(activation=tensor([ 0.0320,  0.0164,  0.0691,  ..., -0.0952,  0.4255, -0.1327]), context='Fifty-six is smaller than seventy-six.', question='# Are these facts correct?', label=' Yes', layer_name='model.layers.15'),
 ActivationSample(activation=tensor([ 0.0300,  0.2755,  0.0297,  ...,  0.0331,  0.3606, -0.1069]), context='Fifty-six is smaller than seventy-six.', question='# May I assume this is correct?', label=' Yes', layer_name='model.layers.16'),
 ActivationSample(activation=tensor([ 0.1466, -0.1212,  0.1019,  ...,  0.0743,  0.1145, -0.0166]), context='Seventy-eight is smaller than eighty-nine.', question='# Can you confirm the accuracy of this?', label=' Yes', layer_name='model.layers.11'),
 ActivationSample(activation=tensor([ 0.0613,  0.1951,  0.0546,  ..., -0.0468,  0.4112, -0.1386]), context='Seventy-nine is smaller than fifty-one.', question='# May I assume this is correct?', label=' No', layer_name='model.layers.16'),
 ActivationSample(activation=tensor([ 0.0

In [56]:
prompts = [
    "The land of the rising sun is",
    "The capital of France is",
    "The Space Needle is a tower in"
]

batch_inputs = prepare_input(
    tokenizer=mt,
    prompts=prompts,
)

In [61]:
batch_inputs

{'input_ids': tensor([[128000,    791,   4363,    315,    279,  16448,   7160,    374],
        [128001, 128001, 128000,    791,   6864,    315,   9822,    374],
        [128000,    791,  11746,  89900,    374,    264,  21970,    304]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [59]:
batch_inputs[1].attention_mask.index(1)

2